# Controller #

Import all neccessary packages

In [1]:
import os
import pandas as pd
import numpy as np

from controllers.inference_controllers.sandbox import Sandbox
from controllers.inference_controllers.generator import Generator
from controllers.inference_controllers.optimiser import Optimiser

from toolboxes.plotting_toolbox.domain import Domain
from toolboxes.inference_toolbox.parameter import Parameter
from toolboxes.inference_toolbox.model import Model
from toolboxes.inference_toolbox.likelihood import Likelihood

current_directory = os.getcwd()
if current_directory != '/project/':
    os.chdir('/project/')

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


Defining simulated data

In [5]:
data_params = {
    'data_type': 'simulated_data',
    'model': {
        'model_select': 'log_gpm_norm',
        'model_params':{
            'H': 10
        },
        'inference_params':{
            'I_y': 1.6,
            'I_z': 1.2,
            'Q': 3e13,
            'sigma': 1
        }
    },
    'domain':{
        'domain_select': 'cone_from_source_z_limited',
        'domain_params': {
            'r': 1000,
            'theta': np.pi/8,
            'source': [0,0,10]
        },
        'resolution': 20
    },
    'noise_dist': 'gaussian',
    'noise_level': 1,
    'output_header': 'Concentration',
}

# data_params = {
#     'data_type': 'normalised_data',
#     'data_select': 'GBR_data',
#     'normaliser_select': 'GBR_normaliser',
#     'normaliser_params':{
#         'experiments_list': [
#             'Exp1',
#             'Exp2',
#             'Exp3',
#             'Exp4',
#             'Exp5',
#             'Exp6',
#             'Exp7',
#             'Exp8',
#             'Exp9',
#             'Exp10',
#             'Exp11',
#             'Exp12',
#         ],
#         'meta_data_select': 'GBR_data_summary',
#         'input_header': 'Concentration'
#     },
#     'log':True,
#     'output_header': 'Concentration',
#     'gridding': [100,100,25]
# }

prior_plots = [
    {
        'Q': [1e12, 5e13],
        'references':{
            'Q' : [2.82e13, 3.11e13, 2.89e13, 4.83e13],
            'labels': ['250m','200m','750m','1000m'] 
        }
    },
    {
        'sigma': [0.001, 4],
    },
    {
        'I_y': [0.0001, 1.7],
        'I_z': [0.0001, 1.7],
        'references':{
            'I_y' : [0.22, 1.6, 0.11, 0.08, 0.06, 0.04],
            'I_z' : [0.2, 1.2, 0.08, 0.06, 0.03, 0.016],
            'labels': ['A','B','C','D','E','F'] 
        }
    }
]

results_name = 'simulated_data'

Defining default parameters and prior plots for gamma priors of the simulated data

In [3]:
default_params = {
    'infered_params':pd.Series({
        'model_params':pd.Series({
            'I_y_and_I_z': Parameter('I_y','I_z', prior_select='log_norm')
                                    .add_prior_param('mu', [0.4,0.4])
                                    .add_prior_param('cov', [[0.8,0],[0,0.8]]),
            'Q': Parameter('Q', prior_select = 'log_norm').add_prior_param('mu', 6e13).add_prior_param('sigma',6e6),
        }),
        'likelihood_params':pd.Series({
            'sigma': Parameter('sigma', prior_select = 'log_norm').add_prior_param('mu', 1.5).add_prior_param('sigma',0.7)
        })
    }),
    'model':Model('log_gpm_norm').add_model_param('H',10),
    'likelihood': Likelihood('gaussian'),
    'sampler': {
        'n_samples': 10000,
        'n_chains': 3,
        'thinning_rate': 1
    }
}

Running a general instance for gamma priors and simulated data

In [4]:
sandbox = Sandbox(results_name=results_name, 
                  data_params=data_params,
                  default_params=default_params)

visualiser = sandbox.run()
visualiser.get_summary()
visualiser.get_traceplot()
visualiser.get_autocorrelations()
visualiser.get_prior_plots(prior_plots)

domain = Domain('cone_from_source_z_limited', resolution=80)
domain.add_domain_param('r', 1000)
domain.add_domain_param('theta', np.pi/8)
domain.add_domain_param('source', [0,0,10])

visualiser.visualise_results(domain = domain, name = 'small_scale_3D_plots', title='Log Concentration of Droplets', log_results=False)
visualiser.animate(name = 'small_scale_3D_plots')

Loading Chain Samples...
Loading Samples...
Traceplot 1 already exists
Traceplot 2 already exists
Traceplot 3 already exists
Autocorrelations plot 1 already exists
Autocorrelations plot 2 already exists
Autocorrelations plot 3 already exists


KeyError: 'I_y_and_I_z'

In [ ]:
default_params = {
    'infered_params':pd.Series({
        'model_params':pd.Series({
            'I_y_and_I_z': Parameter('I_y','I_z', prior_select='log_norm')
                                    .add_prior_param('mu', [0.4,0.4])
                                    .add_prior_param('cov', [[0.8,0],[0,0.8]]),
            'Q': Parameter('Q', prior_select = 'log_norm').add_prior_param('mu', 6e13).add_prior_param('sigma',6e6),
        }),
        'likelihood_params':pd.Series({
            'sigma': Parameter('sigma', prior_select = 'log_norm').add_prior_param('mu', 1.5).add_prior_param('sigma',0.7)
        })
    }),
    'model':Model('log_gpm_norm').add_model_param('H',10),
    'likelihood': Likelihood('gaussian'),
    'sampler': {
        'n_samples': 10000,
        'n_chains': 1,
        'thinning_rate': 1
    }
}

Analysing the sensitivity of the gamma prior simulated data

In [ ]:
generator = Generator(results_name=results_name, 
                  data_params=data_params,
                  default_params=default_params)

analysis_iterations = {
    'parameters_1': ['I_y_and_I_z_mu_0','I_y_and_I_z_mu_1','Q_mu','sigma_mu'],
    'parameters_2': ['I_y_and_I_z_cov_0_0','I_y_and_I_z_cov_1_1','Q_sigma','sigma_sigma'],
    'values_1':
    [
        np.array([1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5]),
        np.array([1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5]),
        np.array([1e9, 1e10, 1e11, 1e12, 1e13, 1e14, 1e15, 1e16, 1e17, 1e18]),
        np.array([0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5])     
    ],

    'values_2':
    [
        np.array([1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5]),
        np.array([1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5]),
        np.array([1e2, 1e3, 1e4, 1e5, 1e6, 1e7, 1e8, 1e9, 1e10, 1e11]),
        np.array([0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]) 
    ],
    'scales_1': ['log','log', 'log', 'linear'],
    'scales_2': ['log','log', 'log', 'linear'],
}

for i in range(len(analysis_iterations['parameters_1'])):
    parameter_1 = analysis_iterations['parameters_1'][i]
    parameter_2 = analysis_iterations['parameters_2'][i]

    print('Working on varying ' + parameter_1 + '...')
    values_1 = analysis_iterations['values_1'][i]
    values_2 = analysis_iterations['values_2'][i]

    scale_1 = analysis_iterations['scales_1'][i]
    scale_2 = analysis_iterations['scales_2'][i]

    inputs = generator.vary_two_parameters(parameter_1, parameter_2, values_1, values_2, scale_1=scale_1, scale_2=scale_2, plot=True)

Working on varying I_y_mu...
Generating instance 1...


sample: 100%|██████████| 12500/12500 [00:02<00:00, 5227.14it/s, 3 steps of size 5.00e-03. acc. prob=0.67]


Generating instance 2...


sample: 100%|██████████| 12500/12500 [00:53<00:00, 233.25it/s, 1023 steps of size 3.16e-06. acc. prob=0.81]


Generating instance 3...


...

Generating instance 56...


sample: 100%|██████████| 12500/12500 [00:04<00:00, 2676.12it/s, 31 steps of size 6.32e-02. acc. prob=0.94]


Generating instance 57...


sample: 100%|██████████| 12500/12500 [00:38<00:00, 325.71it/s, 15 steps of size 3.23e-02. acc. prob=0.93]  


: 

Optimising the hyperparameters for gamma prior simulated data

In [ ]:
optimising_parameters = {
                    'I_y_and_I_z_mu_0': [1e-2, 2],
                    'I_y_and_I_z_mu_1': [1e-2, 2],
                    'I_y_and_I_z_cov_0_0': [1e-2, 2],
                    'I_y_and_I_z_cov_1_1': [1e-2, 2],
                    'Q_mu': [1e12, 1e15],
                    'Q_sigma': [1e5, 1e7],
                    'sigma_mu':[0.2, 2],
                    'sigma_sigma':[0.2, 2]
                }

optimiser = Optimiser(results_name=results_name, 
                  data_params=data_params,
                  default_params=default_params)

# AIC
study = optimiser.run(n_trials=100, optimiser_name='AIC_1', optimising_parameters=optimising_parameters, index_name='aic')
optimiser.get_plots(study)
domain = Domain('cone_from_source_z_limited', resolution=80)
domain.add_domain_param('r', 1000)
domain.add_domain_param('theta', np.pi/8)
domain.add_domain_param('source', [0,0,10])
optimiser.run_best_params(study, domain, name = 'small_scale_3D_plots', prior_plots=prior_plots)

# # RMSE
# study = optimiser.run(n_trials=100, optimiser_name='RMSE_1', optimising_parameters=optimising_parameters, index_name='rmse')
# optimiser.get_plots(study)
# domain = Domain('cone_from_source_z_limited', resolution=80)
# domain.add_domain_param('r', 1000)
# domain.add_domain_param('theta', np.pi/8)
# domain.add_domain_param('source', [0,0,10])
# optimiser.run_best_params(study, domain, name = 'small_scale_3D_plots', prior_plots=prior_plots)

# # BIC
# study = optimiser.run(n_trials=100, optimiser_name='BIC_1', optimising_parameters=optimising_parameters, index_name='bic')
# optimiser.get_plots(study)
# domain = Domain('cone_from_source_z_limited', resolution=80)
# domain.add_domain_param('r', 1000)
# domain.add_domain_param('theta', np.pi/8)
# domain.add_domain_param('source', [0,0,10])
# optimiser.run_best_params(study, domain, name = 'small_scale_3D_plots', prior_plots=prior_plots)

[I 2023-10-17 00:52:53,720] A new study created in RDB with name: AIC_1
sample: 100%|██████████| 12500/12500 [00:02<00:00, 5222.09it/s, 5 steps of size 7.29e-01. acc. prob=0.89]
[I 2023-10-17 00:53:06,108] Trial 0 finished with value: 64.68317413330078 and parameters: {'I_y_mu': 1.7259646638946395, 'I_y_sigma': 1.614344210442248, 'I_z_mu': 1.6832442190290895, 'I_z_sigma': 1.98811013757421, 'Q_mu': 138113822868418.78, 'Q_sigma': 7793230.8623969965, 'sigma_mu': 0.9662126801478261, 'sigma_sigma': 1.1061564087334241}. Best is trial 0 with value: 64.68317413330078.
sample: 100%|██████████| 12500/12500 [00:02<00:00, 5365.77it/s, 5 steps of size 7.36e-01. acc. prob=0.89]
[I 2023-10-17 00:53:17,075] Trial 1 finished with value: 64.69619369506836 and parameters: {'I_y_mu': 0.9924914979420515, 'I_y_sigma': 1.9703274788170275, 'I_z_mu': 0.34612666310080825, 'I_z_sigma': 1.6045283101629289, 'Q_mu': 894983044880725.0, 'Q_sigma': 8493407.7296945, 'sigma_mu': 0.8328451170310072, 'sigma_sigma': 1.2158